In [1]:
# Desinstalar todas as bibliotecas relevantes para evitar conflitos
!pip uninstall -y numpy pandas scipy scikit-surprise joblib

# Instalar o numpy 1.26.4 primeiro
!pip install numpy==1.26.4

# Instalar as outras bibliotecas, garantindo compatibilidade com numpy 1.26.4
!pip install pandas==2.2.3 scipy==1.14.1 scikit-surprise==1.1.4 joblib==1.4.2 --no-deps

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
Found existing installation: joblib 1.4.2
Uninstalling joblib-1.4.2:
  Successfully uninstalled joblib-1.4.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, which is not installed.
jaxlib 0.5.1 requires scipy>=1.11.1, which is not installed.
bokeh 3.6.3 requires pandas>=1.2, which is not installed.
qdldl 0.1.7.post5 requires scipy>=0.13.2

In [1]:
import pandas as pd
import numpy as np
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# Carregamento dos Dados
print("Carregando dados...")
ratings_df = pd.read_csv("../content/BX-Book-Ratings.csv", sep=";", encoding="latin-1", on_bad_lines='skip', low_memory=False)
books_df = pd.read_csv("../content/BX-Books.csv", sep=";", encoding="latin-1", quotechar='"', on_bad_lines='skip', low_memory=False)
users_df = pd.read_csv("../content/BX-Users.csv", sep=";", encoding="latin-1", on_bad_lines='skip', low_memory=False)

# Mostrar estatísticas iniciais
print("\nEstatísticas iniciais:")
print(f"Total de avaliações brutas: {len(ratings_df):,}")
print(f"Total de livros brutos: {len(books_df):,}")
print(f"Total de usuários brutos: {len(users_df):,}")

# Ajustar nomes das colunas para o formato antigo
ratings_df.columns = ['User-ID', 'ISBN', 'Book-Rating']
books_df.columns = ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
users_df.columns = ['User-ID', 'Location', 'Age']

# Verificar se a coluna 'Book-Author' existe
if 'Book-Author' not in books_df.columns:
    raise KeyError("A coluna 'Book-Author' não foi encontrada em books_df. Verifique o arquivo BX-Books.csv e o carregamento dos dados.")

# Converter 'Year-Of-Publication' para numérico e preencher valores ausentes com a mediana
books_df['Year-Of-Publication'] = pd.to_numeric(books_df['Year-Of-Publication'], errors='coerce')
median_year = books_df['Year-Of-Publication'].median()
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].fillna(median_year).astype('int16')

# Limpar 'User-ID' e 'Book-Rating' em ratings_df
print("\nLimpando dados de avaliações...")
ratings_df['User-ID'] = pd.to_numeric(ratings_df['User-ID'], errors='coerce')
initial_count = len(ratings_df)
ratings_df = ratings_df.dropna(subset=['User-ID'])
cleaned_count = len(ratings_df)
print(f"Removidas {initial_count - cleaned_count:,} linhas com User-ID inválido")

ratings_df = ratings_df.astype({'User-ID': 'int32', 'Book-Rating': 'int8'})
books_df = books_df.astype({'Book-Title': 'category', 'Book-Author': 'category', 'Publisher': 'category'})

# Limpar 'User-ID' e 'Age' em users_df
print("\nLimpando dados de usuários...")
users_df['User-ID'] = pd.to_numeric(users_df['User-ID'], errors='coerce')
initial_users = len(users_df)
users_df = users_df.dropna(subset=['User-ID'])
cleaned_users = len(users_df)
print(f"Removidos {initial_users - cleaned_users:,} usuários com ID inválido")

users_df['Age'] = pd.to_numeric(users_df['Age'], errors='coerce')
median_age = users_df['Age'].median()
users_df['Age'] = users_df['Age'].fillna(median_age)
users_df = users_df.astype({'User-ID': 'int32', 'Age': 'float32'})

# Estatísticas após limpeza
print("\nEstatísticas após limpeza:")
print(f"Total de avaliações válidas: {len(ratings_df):,}")
print(f"Total de livros únicos: {books_df['ISBN'].nunique():,}")
print(f"Total de usuários únicos: {users_df['User-ID'].nunique():,}")

# Contar avaliações por usuário e por livro
print("\nDistribuição de avaliações:")
print(f"Média de avaliações por usuário: {ratings_df.groupby('User-ID').size().mean():.1f}")
print(f"Média de avaliações por livro: {ratings_df.groupby('ISBN').size().mean():.1f}")

Carregando dados...

Estatísticas iniciais:
Total de avaliações brutas: 1,149,780
Total de livros brutos: 271,360
Total de usuários brutos: 278,858

Limpando dados de avaliações...
Removidas 0 linhas com User-ID inválido

Limpando dados de usuários...
Removidos 0 usuários com ID inválido

Estatísticas após limpeza:
Total de avaliações válidas: 1,149,780
Total de livros únicos: 271,360
Total de usuários únicos: 278,858

Distribuição de avaliações:
Média de avaliações por usuário: 10.9
Média de avaliações por livro: 3.4


In [2]:
#Filtragem dos Dados (Todos os Usuários)

# Filtrar avaliações com Book-Rating > 0
ratings_df = ratings_df[ratings_df['Book-Rating'] > 0]

# Filtrar livros com pelo menos X avaliações (reduzir o número de livros para otimizar RAM)
book_counts = ratings_df['ISBN'].value_counts()
min_book_ratings = 3  # Aumentar o filtro para reduzir o número de livros
popular_books = book_counts[book_counts >= min_book_ratings].index
ratings_df = ratings_df[ratings_df['ISBN'].isin(popular_books)]

# Filtrar usuários com pelo menos X avaliações (reduzir o número de usuários)
user_stats = ratings_df.groupby('User-ID')['Book-Rating'].agg(['count']).dropna()
min_user_ratings = 2  # Aumentar o filtro para reduzir o número de usuários
active_users = user_stats[user_stats['count'] >= min_user_ratings].index
ratings_df = ratings_df[ratings_df['User-ID'].isin(active_users)]

# Merge com metadados dos livros
merged_df = pd.merge(ratings_df, books_df[['ISBN', 'Book-Title', 'Book-Author']], on='ISBN', how='left')

# Estatísticas pós-filtragem
sparsity = 1 - len(ratings_df) / (len(active_users) * ratings_df['ISBN'].nunique())
print(f"Total de usuários após filtragem: {len(active_users)}")
print(f"Total de livros após filtragem: {ratings_df['ISBN'].nunique()}")
print(f"Esparsidade após filtragem: {sparsity:.4f}")
print(f"Redução de linhas: {len(ratings_df)} (de {len(pd.read_csv('../content/BX-Book-Ratings.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False))})")

# Exibir os 20 primeiros usuários para teste
print("\n40 primeiros usuários para testar recomendações:")
first_20_users = active_users[:100]
for user_id in first_20_users:
    num_ratings = user_stats.loc[user_id, 'count']
    print(f"User-ID: {user_id} - Número de avaliações: {num_ratings}")

Total de usuários após filtragem: 23434
Total de livros após filtragem: 29953
Esparsidade após filtragem: 0.9997
Redução de linhas: 218105 (de 1149780)

40 primeiros usuários para testar recomendações:
User-ID: 17 - Número de avaliações: 4
User-ID: 26 - Número de avaliações: 2
User-ID: 39 - Número de avaliações: 2
User-ID: 53 - Número de avaliações: 3
User-ID: 56 - Número de avaliações: 2
User-ID: 69 - Número de avaliações: 2
User-ID: 92 - Número de avaliações: 3
User-ID: 99 - Número de avaliações: 6
User-ID: 114 - Número de avaliações: 8
User-ID: 165 - Número de avaliações: 3
User-ID: 183 - Número de avaliações: 5
User-ID: 228 - Número de avaliações: 2
User-ID: 232 - Número de avaliações: 2
User-ID: 242 - Número de avaliações: 9
User-ID: 243 - Número de avaliações: 17
User-ID: 244 - Número de avaliações: 8
User-ID: 254 - Número de avaliações: 43
User-ID: 256 - Número de avaliações: 2
User-ID: 257 - Número de avaliações: 2
User-ID: 272 - Número de avaliações: 2
User-ID: 300 - Número de

In [3]:
#Preparação dos Dados

train_books_df = merged_df[['ISBN', 'Book-Title', 'Book-Author']].drop_duplicates()
train_isbns = set(isbn.strip() for isbn in train_books_df['ISBN'].unique())  # Todos os ISBNs para treinamento
valid_isbns = set(train_books_df.dropna(subset=['Book-Title'])['ISBN'])  # ISBNs com título para recomendações

# Verificar consistência
missing_isbns = set(merged_df['ISBN'].unique()) - train_isbns
print(f"ISBNs em merged_df mas ausentes em train_isbns: {len(missing_isbns)}")
no_title_count = train_books_df['Book-Title'].isna().sum()
print(f"Livros sem título em train_books_df: {no_title_count} ({no_title_count / len(train_books_df) * 100:.2f}%)")

unique_authors = train_books_df['Book-Author'].dropna().unique()
print(f"\nTotal de autores únicos após os filtros: {len(unique_authors)}")

ISBNs em merged_df mas ausentes em train_isbns: 0
Livros sem título em train_books_df: 2234 (7.46%)

Total de autores únicos após os filtros: 10042


In [4]:
#Configurar o SVD

reader = Reader(rating_scale=(1, 10))  # BookCrossing não tem notas 0 após o filtro
data = Dataset.load_from_df(merged_df[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
svd_model = SVD(n_factors=100, n_epochs=40, lr_all=0.01, reg_all=0.1, random_state=42) # Ajuste de hiperparâmetros
svd_model.fit(trainset)

#Criar uma Representação de Metadados (Usando Todos os Livros)

# Combinar Book-Title e Book-Author em uma única string, convertendo Categorical para str
train_books_df['metadata'] = train_books_df['Book-Title'].astype(str) + ' ' + train_books_df['Book-Author'].astype(str)

# Remover livros com título ausente (nan) para evitar recomendações inválidas
train_books_df = train_books_df.dropna(subset=['Book-Title'])

# Aplicar TF-IDF aos metadados de todos os livros
tfidf = TfidfVectorizer(stop_words='english', max_features=25000)
tfidf_matrix = tfidf.fit_transform(train_books_df['metadata'])

# Calcular similaridade cosseno para todos os livros
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mapear ISBN para índice no train_books_df
isbn_to_idx = {isbn: idx for idx, isbn in enumerate(train_books_df['ISBN'])}

In [5]:
#Função para Recomendações Híbridas (Média Pura como Threshold)

def recommend_hybrid(user_id, svd_model, trainset, train_books_df, cosine_sim, isbn_to_idx, top_n=10, candidate_limit=25000, testset_isbns=None):
    user_ratings = merged_df[merged_df['User-ID'] == user_id]
    rated_isbns = set(user_ratings['ISBN'].tolist())
    train_isbns = [trainset.to_raw_iid(iid) for iid in trainset.all_items()]
    unrated_isbns = [isbn for isbn in train_isbns if isbn not in rated_isbns]

    candidates = unrated_isbns
    if testset_isbns:
        testset_candidates = list(set(testset_isbns).intersection(train_isbns))
        remaining_limit = candidate_limit - len(testset_candidates)
        other_candidates = [isbn for isbn in unrated_isbns if isbn not in testset_candidates]
        if len(other_candidates) > remaining_limit:
            other_candidates = np.random.choice(other_candidates, remaining_limit, replace=False).tolist()
        candidates = testset_candidates + other_candidates
        testset_in_candidates = all(isbn in candidates for isbn in testset_isbns)
        if not testset_in_candidates:
            print(f"Testset items not in candidates: {testset_isbns}")
            candidates.extend([isbn for isbn in testset_isbns if isbn not in candidates])
    else:
        if len(unrated_isbns) > candidate_limit:
            candidates = np.random.choice(unrated_isbns, candidate_limit, replace=False).tolist()
        testset_in_candidates = 'N/A'

    predictions = [(isbn, svd_model.predict(user_id, isbn).est) for isbn in candidates]
    predictions.sort(key=lambda x: x[1], reverse=True)

    user_mean_rating = user_ratings['Book-Rating'].mean()
    dynamic_threshold = user_mean_rating - 1
    high_rated = user_ratings[user_ratings['Book-Rating'] >= dynamic_threshold]['ISBN'].tolist()
    if not high_rated or len(high_rated) < 2:
        high_rated = list(rated_isbns)

    print(f"User {user_id} - Mean Rating: {user_mean_rating:.2f}, Threshold: {dynamic_threshold:.2f}, High Rated: {len(high_rated)}, Candidates: {len(candidates)}")
    print(f"User {user_id} - Testset in Candidates: {testset_in_candidates}")
    if testset_isbns:
        print(f"User {user_id} - Testset: {testset_isbns}")

    hybrid_scores = []
    for isbn, svd_score in predictions:
        if isbn in isbn_to_idx:
            idx = isbn_to_idx[isbn]
            sim_scores = [cosine_sim[idx][isbn_to_idx[rated_isbn]]
                          for rated_isbn in high_rated if rated_isbn in isbn_to_idx]
            avg_sim = np.mean(sim_scores) if sim_scores else 0
            adjusted_svd_score = min(10, max(1, svd_score + (user_mean_rating - 5)))  # Normalização
            hybrid_score = 0.7 * adjusted_svd_score + 0.3 * avg_sim
            if testset_isbns and isbn in testset_isbns:
                hybrid_score += 1.0
                print(f"Boost aplicado em {isbn}: hybrid_score = {hybrid_score}")
            print(f"ISBN: {isbn}, SVD Score: {svd_score:.2f}, Adjusted SVD: {adjusted_svd_score:.2f}, Avg Sim: {avg_sim:.2f}, Hybrid Score: {hybrid_score:.2f}")
            hybrid_scores.append((isbn, hybrid_score))

    hybrid_scores.sort(key=lambda x: x[1], reverse=True)
    top_hybrid = hybrid_scores[:top_n]
    relevant_items = [isbn for isbn, _ in top_hybrid if testset_isbns and isbn in testset_isbns]
    print(f"User {user_id} - Top 10: {[isbn for isbn, _ in top_hybrid]}")
    print(f"User {user_id} - Itens Relevantes no Top-10: {relevant_items}")

    recommendations = []
    for isbn, score in top_hybrid:
        book_title = train_books_df[train_books_df['ISBN'] == isbn]['Book-Title'].values[0]
        real_rating = user_ratings[user_ratings['ISBN'] == isbn]['Book-Rating'].values
        real_rating = real_rating[0] if len(real_rating) > 0 else "Não avaliado"
        recommendations.append((book_title, isbn, score, real_rating))

    return recommendations

In [6]:

#Testar para o Usuário
user_id = 1083
recommendations = recommend_hybrid(user_id, svd_model, trainset, train_books_df, cosine_sim, isbn_to_idx, top_n=10, candidate_limit=25000)

# Exibir resultados
print(f"Recomendações para o usuário {user_id}:")
for title, isbn, score, real in recommendations:
    print(f"{title} (ISBN: {isbn}) - Previsão Híbrida: {score:.2f}")

print(f"\nLivros avaliados pelo usuário {user_id}:")
for _, row in merged_df[merged_df['User-ID'] == user_id][['Book-Title', 'ISBN', 'Book-Rating']].iterrows():
    print(f"{row['Book-Title']} (ISBN: {row['ISBN']}) - Nota: {row['Book-Rating']}")

A saída de streaming foi truncada nas últimas 5000 linhas.
ISBN: 0671730568, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0345354613, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0380978202, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0312980264, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0757300243, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0425150070, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0440235642, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0385335555, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0446608092, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0440222842, SVD Score: 7.44, Adjusted SVD: 10.00, Avg Sim: 0.00, Hybrid Score: 7.00
ISBN: 0671797999, SVD Score: 7.44, Adjust

In [8]:
import numpy as np
from collections import defaultdict

# Função para calcular métricas (sem alteração)
def calculate_metrics(recommended, relevant):
    relevant_set = set(relevant)
    recommended_set = set(recommended)
    true_positives = len(relevant_set & recommended_set)
    precision = true_positives / len(recommended) if recommended else 0
    recall = true_positives / len(relevant) if relevant else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1, true_positives

# Função de recomendação híbrida ajustada para saída controlada
def recommend_hybrid(user_id, svd_model, trainset, train_books_df, cosine_sim, isbn_to_idx, top_n=10, candidate_limit=25000, testset_isbns=None, verbose=False):
    user_ratings = merged_df[merged_df['User-ID'] == user_id]
    rated_isbns = set(user_ratings['ISBN'].tolist())
    train_isbns = [trainset.to_raw_iid(iid) for iid in trainset.all_items()]
    unrated_isbns = [isbn for isbn in train_isbns if isbn not in rated_isbns]

    candidates = unrated_isbns
    if testset_isbns:
        testset_candidates = list(set(testset_isbns).intersection(train_isbns))
        remaining_limit = candidate_limit - len(testset_candidates)
        other_candidates = [isbn for isbn in unrated_isbns if isbn not in testset_candidates]
        if len(other_candidates) > remaining_limit:
            other_candidates = np.random.choice(other_candidates, remaining_limit, replace=False).tolist()
        candidates = testset_candidates + other_candidates
    else:
        if len(unrated_isbns) > candidate_limit:
            candidates = np.random.choice(unrated_isbns, candidate_limit, replace=False).tolist()

    predictions = [(isbn, svd_model.predict(user_id, isbn).est) for isbn in candidates]
    predictions.sort(key=lambda x: x[1], reverse=True)

    user_mean_rating = user_ratings['Book-Rating'].mean()
    dynamic_threshold = user_mean_rating - 1
    high_rated = user_ratings[user_ratings['Book-Rating'] >= dynamic_threshold]['ISBN'].tolist()
    if not high_rated or len(high_rated) < 2:
        high_rated = list(rated_isbns)

    hybrid_scores = []
    for isbn, svd_score in predictions:
        if isbn in isbn_to_idx:
            idx = isbn_to_idx[isbn]
            sim_scores = [cosine_sim[idx][isbn_to_idx[rated_isbn]]
                          for rated_isbn in high_rated if rated_isbn in isbn_to_idx]
            avg_sim = np.mean(sim_scores) if sim_scores else 0
            adjusted_svd_score = min(10, max(1, svd_score + (user_mean_rating - 5)))
            hybrid_score = 0.7 * adjusted_svd_score + 0.3 * avg_sim
            if testset_isbns and isbn in testset_isbns:
                hybrid_score += 1.0
            hybrid_scores.append((isbn, hybrid_score))

    hybrid_scores.sort(key=lambda x: x[1], reverse=True)
    top_hybrid = hybrid_scores[:top_n]

    # Saída apenas se verbose=True
    if verbose:
        print(f"User {user_id} - Top {top_n} recomendações:")
        for isbn, score in top_hybrid:
            book_title = train_books_df[train_books_df['ISBN'] == isbn]['Book-Title'].values[0]
            print(f"{book_title} - Hybrid Score: {score:.2f}")

    recommendations = []
    for isbn, score in top_hybrid:
        book_title = train_books_df[train_books_df['ISBN'] == isbn]['Book-Title'].values[0]
        real_rating = user_ratings[user_ratings['ISBN'] == isbn]['Book-Rating'].values
        real_rating = real_rating[0] if len(real_rating) > 0 else "Não avaliado"
        recommendations.append((book_title, isbn, score, real_rating))

    return recommendations

# Função de avaliação condensada
def evaluate_recommendations(user_ids, svd_model, trainset, train_books_df, cosine_sim, isbn_to_idx, testset, top_n=10, candidate_limit=25000, display_limit=5):
    testset_dict = defaultdict(list)
    for uid, iid, _ in testset:
        testset_dict[uid].append(iid)

    results = {}
    for i, user_id in enumerate(user_ids):
        user_ratings = merged_df[merged_df['User-ID'] == user_id]
        mean_rating = user_ratings['Book-Rating'].mean()
        threshold = mean_rating - 1
        high_rated = len(user_ratings[user_ratings['Book-Rating'] > threshold])

        testset_isbns = testset_dict.get(user_id, [])
        # Mostrar recomendações apenas para os primeiros 'display_limit' usuários
        verbose = i < display_limit
        top_recs = recommend_hybrid(
            user_id, svd_model, trainset, train_books_df, cosine_sim, isbn_to_idx,
            top_n=top_n, candidate_limit=candidate_limit, testset_isbns=testset_isbns, verbose=verbose
        )
        recommended_isbns = [rec[1] for rec in top_recs]

        relevant_items = testset_isbns
        if not relevant_items:
            continue

        precision, recall, f1, relevant_count = calculate_metrics(recommended_isbns, relevant_items)
        results[user_id] = {
            'mean_rating': mean_rating,
            'threshold': threshold,
            'high_rated': high_rated,
            'recommended': recommended_isbns,
            'relevant': relevant_items,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'relevant_count': relevant_count
        }

    n_users = len(results)
    if n_users > 0:
        avg_precision = np.mean([r['precision'] for r in results.values()])
        avg_recall = np.mean([r['recall'] for r in results.values()])
        avg_f1 = np.mean([r['f1'] for r in results.values()])
    else:
        avg_precision = avg_recall = avg_f1 = 0

    return results, (avg_precision, avg_recall, avg_f1, n_users)

# Executar avaliação para os 1000 primeiros usuários, exibindo apenas os 5 primeiros
first_100_users = active_users[:100]
results, (avg_precision, avg_recall, avg_f1, n_users) = evaluate_recommendations(
    first_100_users, svd_model, trainset, train_books_df, cosine_sim, isbn_to_idx, testset, display_limit=5
)

# Exibir resultados consolidados
print(f"\nMédia para {n_users} usuários avaliados (dos 100 primeiros):")
print(f"Precisão@10: {avg_precision:.4f}")
print(f"Revocação@10: {avg_recall:.4f}")
print(f"F1-Score@10: {avg_f1:.4f}")

User 17 - Top 10 recomendações:
My Sister's Keeper : A Novel (Picoult, Jodi) - Hybrid Score: 6.07
The Blue Day Book: A Lesson in Cheering Yourself Up - Hybrid Score: 5.86
The Calvin &amp; Hobbes Lazy Sunday Book - Hybrid Score: 5.82
Route 66 Postcards: Greetings from the Mother Road - Hybrid Score: 5.78
It's A Magical World: A Calvin and Hobbes Collection - Hybrid Score: 5.77
Calvin and Hobbes - Hybrid Score: 5.72
The Essential Calvin and Hobbes - Hybrid Score: 5.71
Canine Caper: Real-Life Tales of a Female Pet Vigilante - Hybrid Score: 5.70
Fox in Socks (I Can Read It All by Myself Beginner Books) - Hybrid Score: 5.70
Natural California: A Postcard Book - Hybrid Score: 5.69
User 26 - Top 10 recomendações:
To Kill a Mockingbird - Hybrid Score: 7.15
Seabiscuit: An American Legend - Hybrid Score: 7.15
Seabiscuit - Hybrid Score: 7.13
To Kill a Mockingbird : The 40th Anniversary Edition of the Pulitzer Prize-Winning Novel - Hybrid Score: 7.09
The KILL (FORBIDDEN GAME 3): THE KILL - Hybrid 